##### Imports

In [1]:
import sys
from pathlib import Path
import warnings
from torch.utils.data import Dataset

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import pickle
from pathlib import Path
from scr.dataset import *
# from scr.game import *
from scr.feature_engineering import *
# from scr.plot_utils import *
import gc
from scr.utils import print_scenarios

from pathlib import Path
import random
from collections import Counter, defaultdict
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset

from scr.utils import read_words, save_words_to_file

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words

set_seed(42)

import torch
import torch.nn as nn
from pathlib import Path
import random

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Read and Shuffle Word List
word_list = read_words('data/words_250000_train.txt') # , limit=10000)
# word_list = read_words('data/250k.txt', limit=10000)

random.shuffle(word_list)

# Calculate Frequencies and Max Word Length
word_frequencies = calculate_word_frequencies(word_list)
char_frequency = calculate_char_frequencies(word_list)
max_word_length = max(len(word) for word in word_list)

##### Data Path

In [2]:
import shutil
from pathlib import Path

NUM_STRATIFIED_SAMPLES = 5_000 # This will be overwritten by Papermill

# Define the base directory and the paths for training and validation parquet files
base_dataset_dir = Path("/media/sayem/510B93E12554BBD1/dataset/")

stratified_samples_dir = base_dataset_dir / str(NUM_STRATIFIED_SAMPLES)

parquet_path = stratified_samples_dir / 'parquets'
# parquet_valid_path = stratified_samples_dir / 'valid_parquets'

# Function to delete and recreate a directory
def recreate_directory(path):
    if path.exists():
        shutil.rmtree(path)  # Delete the directory and its contents
    path.mkdir(parents=True)  # Create the directory

# Recreate the train and valid directories
recreate_directory(parquet_path)
# recreate_directory(parquet_valid_path)

# print(f"Directories '{parquet_train_path}' and '{parquet_valid_path}' have been recreated.")

print(f"Directories '{parquet_path}' have been recreated.")

Directories '/media/sayem/510B93E12554BBD1/dataset/5000/parquets' have been recreated.


In [3]:
len(word_list)

227300

##### Testing Data

In [4]:
# Define the total number of words and the number of test samples
from scr.custom_sampler import *
NUM_TEST_SAMPLES = 10_000

# Assuming 'word_list' contains the 250,000 words
# First, separate 10,000 words for the final testing set
testing_words = stratified_sample_by_length_and_uniqueness(
    word_list, 
    NUM_TEST_SAMPLES
)

# Define the file path for saving the testing words
testing_words_file_path = stratified_samples_dir / "testing_words.txt"

# Save the testing words to a file
with open(testing_words_file_path, 'w') as file:
    for word in testing_words:
        file.write(word + '\n')

print(f"Testing words saved in {testing_words_file_path}")

# Now, remove these testing samples from the original word list
remaining_words = [word for word in word_list if word not in testing_words]

Testing words saved in /media/sayem/510B93E12554BBD1/dataset/5000/testing_words.txt


##### Stratified Sample Generation

In [5]:
## we are taking starified samples from train_words

from scr.custom_sampler import \
    stratified_sample_by_length_and_frequency, \
        stratified_sample_by_length, stratified_sample_by_length_and_uniqueness

print(f'Number of Statrified samples: {NUM_STRATIFIED_SAMPLES}')

# sampled_words_by_length_and_frequency \
#     = stratified_sample_by_length_and_frequency(train_words, \
#     word_frequencies, \
#     NUM_STRATIFIED_SAMPLES)

sampled_words_by_length = stratified_sample_by_length_and_uniqueness(remaining_words, \
    NUM_STRATIFIED_SAMPLES)

print(len(sampled_words_by_length))

Number of Statrified samples: 5000
5039


##### Intial State Simulation Testing

In [6]:
from scr.game import *

In [7]:
# word = "mississippi"
word = "mythopoetize"
# word = "cat"

initial_states = process_word_for_six_states(word)

initial_states

{'allMasked': '____________',
 'early': '_y__________',
 'quarterRevealed': '_y_h________',
 'midRevealed': '_y_h___e__ze',
 'midLateRevealed': 'myth___et_ze',
 'lateRevealed': 'mytho_oetize',
 'nearEnd': 'mytho_oetize'}

In [8]:
len(initial_states)

7

In [9]:
initial_states

{'allMasked': '____________',
 'early': '_y__________',
 'quarterRevealed': '_y_h________',
 'midRevealed': '_y_h___e__ze',
 'midLateRevealed': 'myth___et_ze',
 'lateRevealed': 'mytho_oetize',
 'nearEnd': 'mytho_oetize'}

##### Dataset Generation: Simulation

In [10]:
word = "mississippi"

initial_states = process_word_for_six_states(word)

# print(initial_states)
# Print generated initial states
print("Generated Initial States: ", initial_states)

Generated Initial States:  {'allMasked': '___________', 'early': '________pp_', 'quarterRevealed': '________pp_', 'midRevealed': '_i__i__ippi', 'midLateRevealed': '_ississippi', 'lateRevealed': '_ississippi', 'nearEnd': '_ississippi'}


In [11]:
from scr.game import simulate_game_progress, \
    play_game_with_a_word, process_word

# Example word and initial state
# Example usage
word = "mississippi"
# word = "cat"

initial_states = process_word_for_six_states(word)

# print(initial_states)
# Print generated initial states
print("Generated Initial States:")
i = 1 
for state_name, initial_state in initial_states.items():
    # Simulate the game
    print(initial_state)
    print(f"{i}: For initial state: {initial_state}")
    won, game_progress = simulate_game_progress(
        model=None,  # Assuming model is not used in this example
        word=word, 
        initial_state=initial_state, 
        char_frequency={},  # Assuming char_frequency is not used in this example
        max_word_length=len(word), 
        device=None,  # Assuming device is not used in this example
        max_attempts=6, 
        normalize=True,
        difficulty="medium", 
        outcome_preference='win'
    )

    # Display game progress
    for step in game_progress:
        print(f"Guessed: '{step[0]}', New State: '{step[1]}', Correct: {step[2]}")

        # break

    i+=1

    # break

    # print("Game Result:", "Won" if won else "Lost")

Generated Initial States:
___________
1: For initial state: ___________
Guessed: 'h', New State: '___________', Correct: False
Guessed: 'j', New State: '___________', Correct: False
Guessed: 'l', New State: '___________', Correct: False
Guessed: 'i', New State: '_i__i__i__i', Correct: True
Guessed: 's', New State: '_ississi__i', Correct: True
Guessed: 'p', New State: '_ississippi', Correct: True
Guessed: 'm', New State: 'mississippi', Correct: True
_i__i__i__i
2: For initial state: _i__i__i__i
Guessed: 's', New State: '_ississi__i', Correct: True
Guessed: 'e', New State: '_ississi__i', Correct: False
Guessed: 'm', New State: 'mississi__i', Correct: True
Guessed: 'p', New State: 'mississippi', Correct: True
_i__i__i__i
3: For initial state: _i__i__i__i
Guessed: 'q', New State: '_i__i__i__i', Correct: False
Guessed: 's', New State: '_ississi__i', Correct: True
Guessed: 'm', New State: 'mississi__i', Correct: True
Guessed: 'v', New State: 'mississi__i', Correct: False
Guessed: 'o', New St

##### Writing Parquet

In [12]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Assuming the function 'process_word_for_six_states' is defined elsewhere
from scr.game import simulate_game_progress, process_word_for_six_states

def process_batch_to_parquet(batch, file_path, start_game_counter):
    game_counter = start_game_counter
    data_for_parquet = []

    for game_data in batch:
        word, state_name, initial_state, difficulty, outcome, won, game_progress = game_data
        if not game_progress:
            continue

        final_state = game_progress[-1][1]
        guessed_states = [initial_state] + [state for _, state, _ in game_progress]
        guessed_letters = [letter for letter, _, _ in game_progress]

        data_for_parquet.append({
            'game_id': game_counter,
            'word': word,
            'initial_state': initial_state,
            'final_state': final_state,
            'guessed_states': ','.join(guessed_states[:-1]),
            'guessed_letters': ','.join(guessed_letters),
            'game_state': state_name,
            'difficulty': difficulty,
            'outcome': outcome,
            'word_length': len(word),
            'won': won
        })

        game_counter += 1

    df = pd.DataFrame(data_for_parquet)
    table = pa.Table.from_pandas(df)
    pq.write_to_dataset(table, root_path=file_path, compression='snappy')

    return game_counter

def generate_batch_for_word(word):
    batch = []
    initial_states = process_word_for_six_states(word)
    games_generated = 0  # Counter for games generated

    for state_name, initial_state in initial_states.items():
        for difficulty in ["easy", "medium", "hard"]:
            for outcome in ["win", "lose"]:
                won, game_progress = simulate_game_progress(
                    model=None, word=word, initial_state=initial_state,
                    char_frequency={}, max_word_length=len(word),
                    device=None, max_attempts=6, normalize=True,
                    difficulty=difficulty, outcome_preference=outcome
                )
                batch.append((word, state_name, initial_state, \
                    difficulty, outcome, won, game_progress))
                games_generated += 1  # Increment game counter

    return batch, games_generated

def main_execution(words, parquet_path):
    game_counter = 0
    games_per_word = len(initial_states) * 3 * 2  # 7 states, 3 difficulties, 2 outcomes

    for word in tqdm(words, desc="Processing Words"):
        batch, games_generated = generate_batch_for_word(word)

        # Update game count
        game_counter += games_generated

        # Process and save batch
        process_batch_to_parquet(batch, parquet_path, game_counter)

    print(f"Total games processed: {game_counter}")

# Execute the main function
main_execution(sampled_words_by_length, parquet_path)

Processing Words:   2%|▏         | 87/5039 [00:00<00:17, 287.28it/s]

Processing Words: 100%|██████████| 5039/5039 [00:16<00:00, 308.12it/s]

Total games processed: 211638


##### Checking the dataset 

In [13]:
import pandas as pd
from pathlib import Path


# Find all Parquet files in the directory
parquet_files = list(parquet_path.glob('*.parquet'))

if parquet_files:
    total_game_sequences = 0

    # Iterate over each file and sum the number of game sequences
    for file in parquet_files:
        df = pd.read_parquet(file)
        total_game_sequences += len(df)

    print(f"Total number of game sequences across all files: {total_game_sequences}")
else:
    print("No Parquet files found in the specified directory.")


Total number of game sequences across all files: 211602


In [14]:
import pandas as pd
import glob
from pathlib import Path

# Use glob to find all Parquet files in the folder
parquet_files = parquet_path.glob('*.parquet')

# Read and concatenate all Parquet files into a single DataFrame
df = pd.concat([pd.read_parquet(file) for file in \
    parquet_files], ignore_index=True)

# # Display the first few rows of the DataFrame
# print(df.head())

# Get the total number of rows (games) in the DataFrame
total_games = len(df)
print(f"Total number of games in the dataset: {total_games}")

# Additional checks and summary statistics
print("Null values in each column:")
print(df.isnull().sum())

print("\nSummary statistics:")
print(df.describe())

# Count the number of unique words or game states
unique_words = df['word'].nunique()
print(f"\nNumber of unique words: {unique_words}")

# Inspect the distribution of game outcomes, difficulties, etc.
print("\nOutcome distribution:")
print(df['outcome'].value_counts())

print("\nDifficulty distribution:")
print(df['difficulty'].value_counts())

# Word length distribution
print("\nWord Length Distribution:")
print(df['word_length'].value_counts())

Total number of games in the dataset: 211602
Null values in each column:
game_id            0
word               0
initial_state      0
final_state        0
guessed_states     0
guessed_letters    0
game_state         0
difficulty         0
outcome            0
word_length        0
won                0
dtype: int64

Summary statistics:
             game_id    word_length
count  211602.000000  211602.000000
mean   105843.131525       9.411064
std     61085.440017       3.100986
min        42.000000       1.000000
25%     52942.250000       7.000000
50%    105842.500000       9.000000
75%    158742.750000      11.000000
max    211679.000000      29.000000

Number of unique words: 5039

Outcome distribution:
outcome
win     105801
lose    105801
Name: count, dtype: int64

Difficulty distribution:
difficulty
easy      70534
medium    70534
hard      70534
Name: count, dtype: int64

Word Length Distribution:
word_length
9     28602
8     28140
10    24906
7     23982
11    21042
6     18102

In [15]:
import pandas as pd

# # Replace this with the path to your Parquet file
# parquet_file_path = 'path/to/your/HangmanData.parquet'

# Read the Parquet file
df = pd.read_parquet(parquet_path)

# Display the first few rows of the DataFrame
df

,game_id,word,initial_state,final_state,guessed_states,guessed_letters,game_state,difficulty,outcome,word_length,won
0,203448,wibblewabble,____________,wibblewabble,"____________,____________,____________,_____e_...","z,u,e,w,a,b,l,i",allMasked,easy,win,12,True
1,203449,wibblewabble,____________,wibblewabble,"____________,_i__________,_i___e_____e,wi___ew...","i,e,w,a,l,b",allMasked,easy,lose,12,True
2,203450,wibblewabble,____________,____l_____l_,"____________,____________,____________,_______...","u,f,g,c,l,d,j",allMasked,medium,win,12,False
3,203451,wibblewabble,____________,wibblewabble,"____________,__bb____bb__,__bbl___bbl_,__bble_...","b,l,e,v,i,y,a,o,w",allMasked,medium,lose,12,True
4,203452,wibblewabble,____________,__bb____bb__,"____________,____________,____________,_______...","x,h,u,c,b,o,y",allMasked,hard,win,12,False
...,...,...,...,...,...,...,...,...,...,...,...
211597,40315,bellbird,be__bird,bellbird,be__bird,l,nearEnd,easy,lose,8,True
211598,40316,bellbird,be__bird,bellbird,"be__bird,be__bird","s,l",nearEnd,medium,win,8,True
211599,40317,bellbird,be__bird,bellbird,be__bird,l,nearEnd,medium,lose,8,True
211600,40318,bellbird,be__bird,bellbird,be__bird,l,nearEnd,hard,win,8,True


In [16]:
# Get the total number of rows (games) in the DataFrame
total_games = len(df)

print(f"Total number of games in the dataset: {total_games}")

# Additional checks you might want to perform:
# - Check for any null values or anomalies in the data
print(df.isnull().sum())

# - Get a summary of the DataFrame
print(df.describe())

# - Count the number of unique words or game states
unique_words = df['word'].nunique()
print(f"Number of unique words: {unique_words}")

# - Inspect the distribution of game outcomes, difficulties, etc.
print(df['outcome'].value_counts())
print(df['difficulty'].value_counts())

Total number of games in the dataset: 211602
game_id            0
word               0
initial_state      0
final_state        0
guessed_states     0
guessed_letters    0
game_state         0
difficulty         0
outcome            0
word_length        0
won                0
dtype: int64
             game_id    word_length
count  211602.000000  211602.000000
mean   105843.131525       9.411064
std     61085.440017       3.100986
min        42.000000       1.000000
25%     52942.250000       7.000000
50%    105842.500000       9.000000
75%    158742.750000      11.000000
max    211679.000000      29.000000
Number of unique words: 5039
outcome
win     105801
lose    105801
Name: count, dtype: int64
difficulty
easy      70534
medium    70534
hard      70534
Name: count, dtype: int64


##### Dataset Creation and train test split

In [17]:
# Create datasets directly from the saved parquet files
hangman_dataset = HangmanDataset(parquet_path)
# valid_dataset = HangmanDataset(parquet_valid_path)

In [18]:
hangman_dataset.get_group_info()

{('easy', 'win'): 35267,
 ('easy', 'lose'): 35267,
 ('medium', 'win'): 35267,
 ('medium', 'lose'): 35267,
 ('hard', 'win'): 35267,
 ('hard', 'lose'): 35267}